In [1]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

import neural_network_methods as nn

In [2]:
data_total = np.loadtxt('diabetes_dataset_total.csv', str)
np.version.version
# data = np.loadtxt('diabetes_dataset.csv', str)
# data_test = np.loadtxt('diabetes_test_set.csv', str)

'1.26.4'

In [3]:
def construct_X(data):
    
    n = len(data)
    num_features = 0
    
    for i in range(len(data[0])):
        if(data[0][i] == ","):
            num_features += 1
    
    X = np.zeros((n, num_features))
    for i in range(n):
        arr = data[i].split(",")
        for j in range(num_features):
            X[i][j] = arr[j]
        
    return X
    

def construct_Y(data):
    
    n = len(data)
    
    Y = np.zeros(n)
    for i in range(n):
        Y[i] = data[i][-1]
        
    return Y

def normalize(X):
    num_features = X.shape[0]
    for i in range(num_features):
        X[i] = (X[i]-np.mean(X[i]))/np.std(X[i])
        
    return X

In [4]:
def split_train_test(data, num_train):
    
    train = data[0:num_train]
    test  = data[num_train:]
    
    return train, test

In [5]:
X_total = construct_X(data_total)
Y_total = construct_Y(data_total)
print(X_total.shape)

(768, 8)


In [6]:
X_total = construct_X(data_total)
Y_total = construct_Y(data_total)

X, test_X, Y, test_Y = train_test_split(X_total, Y_total, train_size = .912, random_state = 2)
# train, test = split_train_test(data_total, 700)
print(X.shape, test_X.shape)

(700, 8) (68, 8)


In [7]:
num_features = len(X_total[1])
m = len(X)

layer_array = [1]
activation_array = ['sigmoid']

In [8]:
def neural_network(X, Y, layer_array, activation_array, lamb, optimizer = 'normal', num_iter=10000):
    
    L = len(layer_array)
    cost_array = np.zeros(int(num_iter/1000))
    alpha = .0001
    t = 0
    beta_momentum = .95
    beta_adam_1 = .92
    beta_adam_2 = .998
    eps = 1e-8
    params = nn.initialize_params(layer_array, X, 0.011)
#     print(params)
    if(optimizer == 'momentum'):
        v = nn.initialize_optimizer(params, layer_array, optimizer)
    elif(optimizer == 'Adam'):    
        v, s = nn.initialize_optimizer(params, layer_array, optimizer)

    for i in range(num_iter):
        A, stored_Al_Zl = nn.forward_prop(params, activation_array, layer_array, X)
        
        cost = nn.comp_cost(Y, stored_Al_Zl[f"A{L}"], params, L, lamb, regularization = True)
        
        grads = nn.back_prop(layer_array, activation_array, stored_Al_Zl, params, X, Y, lamb)
        
        if(optimizer == 'normal'):
            params = nn.update_params(params, grads, alpha, layer_array, 0, 0, t, 0, 0, 0, eps, optimizer)
        elif(optimizer == 'momentum'):
            params, v = nn.update_params(params, grads, alpha, layer_array, v, 0, 0, beta_momentum, 0, 0, eps, optimizer)
        elif(optimizer == 'Adam'):
            params, v, s, v_fixed, s_fixed = nn.update_params(params, grads, alpha, layer_array, v, s, t, beta_momentum, beta_adam_1, beta_adam_2, eps, optimizer)
            t = t+1
            print(grads)
        #         params = nn.update_params(params, grads, .1, layer_array)
        
        if(i%1000 == 0):
            print(i, cost)
            index = i%1000
            cost_array[index] = cost
            
    return params, cost_array

In [9]:
print(X.shape, Y.shape)

(700, 8) (700,)


In [10]:
params, cost_array = neural_network(X.T, Y, layer_array, activation_array, 0.2, optimizer = 'normal', num_iter=1000000)

0 0.9346141457112993
1000 0.6149758428487735
2000 0.6118193813039464
3000 0.6095610636089391
4000 0.6078850743330466
5000 0.6066254938464264
6000 0.6056675110440114
7000 0.6049287960746768
8000 0.6043498952770534
9000 0.6038877523689941
10000 0.6035111097919232
11000 0.6031972192343815
12000 0.6029294843579693
13000 0.602695768422153
14000 0.6024871762933656
15000 0.6022971753852617
16000 0.602120959325609
17000 0.6019549859627865
18000 0.601796640992977
19000 0.6016439923978242
20000 0.6014956107365295
21000 0.601350437338221
22000 0.6012076874327839
23000 0.6010667788311569
24000 0.600927279332652
25000 0.6007888678873453
26000 0.6006513058804904
27000 0.6005144158778245
28000 0.6003780658783265
29000 0.6002421576376814
30000 0.6001066180039213
31000 0.5999713924841725
32000 0.5998364404654029
33000 0.5997017316622559
34000 0.5995672434758511
35000 0.5994329590292418
36000 0.5992988657057418
37000 0.5991649540611205
38000 0.599031217013862
39000 0.5988976492422965
40000 0.59876424673

322000 0.5666677995434385
323000 0.5665714587780579
324000 0.566475227960868
325000 0.566379106959616
326000 0.5662830956421925
327000 0.5661871938766326
328000 0.5660914015311148
329000 0.5659957184739617
330000 0.565900144573639
331000 0.5658046796987565
332000 0.5657093237180665
333000 0.5656140765004648
334000 0.5655189379149902
335000 0.5654239078308242
336000 0.5653289861172913
337000 0.5652341726438586
338000 0.5651394672801356
339000 0.5650448698958739
340000 0.5649503803609677
341000 0.5648559985454529
342000 0.564761724319508
343000 0.5646675575534525
344000 0.5645734981177484
345000 0.5644795458829988
346000 0.5643857007199479
347000 0.5642919624994819
348000 0.5641983310926281
349000 0.5641048063705544
350000 0.56401138820457
351000 0.5639180764661246
352000 0.5638248710268088
353000 0.5637317717583538
354000 0.5636387785326311
355000 0.5635458912216528
356000 0.5634531096975705
357000 0.5633604338326766
358000 0.5632678634994028
359000 0.5631753985703213
360000 0.563083038

639000 0.5409955264540662
640000 0.5409281248381723
641000 0.540860797865828
642000 0.540793545444976
643000 0.5407263674836704
644000 0.5406592638900765
645000 0.5405922345724705
646000 0.5405252794392396
647000 0.5404583983988813
648000 0.5403915913600047
649000 0.5403248582313287
650000 0.5402581989216838
651000 0.54019161334001
652000 0.5401251013953576
653000 0.540058662996888
654000 0.5399922980538718
655000 0.5399260064756907
656000 0.539859788171835
657000 0.5397936430519066
658000 0.5397275710256154
659000 0.5396615720027823
660000 0.5395956458933373
661000 0.539529792607319
662000 0.5394640120548772
663000 0.5393983041462693
664000 0.5393326687918634
665000 0.5392671059021352
666000 0.5392016153876704
667000 0.5391361971591629
668000 0.5390708511274164
669000 0.5390055772033421
670000 0.5389403752979605
671000 0.538875245322401
672000 0.5388101871879004
673000 0.5387452008058047
674000 0.5386802860875671
675000 0.5386154429447503
676000 0.5385506712890239
677000 0.53848597103

957000 0.5228792713383552
958000 0.5228315161070198
959000 0.5227838113322472
960000 0.52273615695242
961000 0.5226885529060011
962000 0.5226409991315334
963000 0.5225934955676402
964000 0.5225460421530241
965000 0.5224986388264682
966000 0.5224512855268347
967000 0.5224039821930663
968000 0.522356728764184
969000 0.5223095251792895
970000 0.5222623713775633
971000 0.5222152672982653
972000 0.5221682128807341
973000 0.5221212080643883
974000 0.5220742527887247
975000 0.5220273469933192
976000 0.5219804906178264
977000 0.5219336836019798
978000 0.5218869258855917
979000 0.5218402174085525
980000 0.5217935581108314
981000 0.5217469479324757
982000 0.5217003868136108
983000 0.5216538746944404
984000 0.5216074115152461
985000 0.521560997216388
986000 0.5215146317383037
987000 0.5214683150215085
988000 0.5214220470065954
989000 0.521375827634235
990000 0.5213296568451752
991000 0.5212835345802419
992000 0.5212374607803382
993000 0.5211914353864434
994000 0.5211454583396149
995000 0.52109952

In [11]:
preds_train,A_final, accuracy = nn.predict(params, activation_array, layer_array, X.T, Y)

In [12]:
print('Training set accuracy:',accuracy)

Training set accuracy: 0.7671428571428571


In [13]:
preds_test,A_final, accuracy = nn.predict(params, activation_array, layer_array, test_X.T, test_Y)
print('Test set accuracy:',accuracy)

Test set accuracy: 0.7205882352941176


In [14]:
print('F1 score:', f1_score(test_Y, preds_test[0]))
print('Precision score:', precision_score(test_Y, np.int64(preds_test)[0]))
print('Recall score:', recall_score(test_Y, np.int64(preds_test)[0]))

F1 score: 0.5777777777777778
Precision score: 0.6842105263157895
Recall score: 0.5


Selected Cases (700 train, 68 test): 

       Layers | Initial Factor|beta_1| alpha | lambda | train | test  | f1    | precision | recall| layer breakdown| Runs
       5      |   0.07        |0.00  | .1    | 0.2    | .752  | .721  | .5366 | .7333     | .4231 |  (20,10,5,3,1) | 20000
       5      |   0.07        |0.00  | .025  | 0.5    | .867  | .750  | .6909 | .6552     | .7308 |  (20,10,5,3,1) | 50000
       5      |   0.07        |0.00  | .010  | 0.5    | .849  | .750  | .7213 | .6286     | .8462 |  (20,10,5,3,1) | 50000
       
       **Momentum**
       5      |   0.07        |0.91  | .1    | 0.20   | .851  | .750  | .6909 | .6552     | .7308 |  (10,7,5,3,1)  | 50000
       5      |   0.07        |0.95  | .1    | 0.20   | .909  | .794  | .7407 | .7143     | .7692 |  (20,10,5,3,1) | 20000
       5      |   0.07        |0.96  | .1    | 0.20   | .959  | .662  | .5818 | .5517     | .6154 |  (20,10,5,3,1) | 20000
       5      |   0.07        |0.97  | .1    | 0.15   | .904  | .765  | .6923 | .6923     | .6923 |  (20,10,5,3,1) | 20000

Special thanks to DeepLearning.ai for the coursework/syntax help on the NN structure codes.

Retrieved from DeepLearning.AI (2024, December 25). 

Logistic Regression with a Neural Network Mindset. Retrieved from C1_W2 Graded Assignment 1 in the Deep Learning Specialization

Planar Data Classification with One Hidden layer. Retrieved from C1_W3 Graded Assignment 1 in the Deep Learning Specialization

Build your Deep Neural Network: Step by Step. Retrieved from C1_W4 Graded Assignment 1 in the Deep Learning Specialization

Deep Neural Network - Application. Retrieved from C1_W4 Graded Assignment 3 in the Deep Learning Specialization

Gradient Checking. Retrieved from C2_W1 Graded Assignment 3 in the Deep Learning Specialization

Optimization methods. Retrieved from C2_W2 Graded Assignment 1 in the Deep Learning Specialization